In [20]:
import pandas as pd
import re
import jieba as jb
from keras.preprocessing.text import Tokenizer

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [2]:
df = pd.read_excel ('測試label.xlsx')

In [3]:
df=df[['類別','句子']]

In [4]:
print("数据总量: %d ." % len(df))
df.sample(10)

数据总量: 241 .


,類別,句子
9,Lobulated contour of liver parenchyma favoring...,lobulated contour with intrahepatic vessel tor...
80,"Mass lesion with early enhancement, early wash...",an early enhance nodule size with early washou...
3,Atrophy change of liver,local liver atrophic change
143,"Mass lesion with early enhancement, early wash...",an early enhance and early washoy tumor size i...
20,Lobulated contour of liver parenchyma favoring...,mild lobulated contour of liver contour
239,"Mass lesion with early enhancement, persistent...",* favor hemangioma in lls and local.
213,Mass lesion with no early enhancement but earl...,one early washout liver nodule about size in l...
108,"Mass lesion with early enhancement, early wash...",local-size liver nodule size with early enhanc...
109,"Mass lesion with early enhancement, early wash...","dwi, post enhance show early enhance and early..."
2,Atrophy change of liver,atrophic change of local liver


In [6]:
df['label_id'] = df['類別'].factorize()[0]
cat_id_df = df[['類別', 'label_id']].drop_duplicates().sort_values('label_id').reset_index(drop=True)
cat_to_id = dict(cat_id_df.values)
id_to_cat = dict(cat_id_df[['label_id', '類別']].values)
df.sample(10)
#0-23類

,類別,句子,label_id
222,Mass lesion with no early enhancement but earl...,"and washout, favor hcc",22
175,"Mass lesion with early enhancement, early wash...",multiple early enhance liver nodule in local-s...,21
74,"Mass lesion with early enhancement, early wash...",multiple early enhance liver nodule cluster ov...,21
53,Hypodense lesion favoring post TAE/TACE necrot...,Hypodense lesion favoring post TAE/TACE necrot...,13
12,Lobulated contour of liver parenchyma favoring...,"* mild lobulated contour of liver, suggestive ...",2
30,Diffuse low attenuation of liver parenchyma fa...,Diffuse low attenuation of liver parenchyma fa...,4
183,"Mass lesion with early enhancement, early wash...",multiple early enhance liver nodule in local l...,21
168,"Mass lesion with early enhancement, early wash...",an early enhance and early washout tumor size ...,21
110,"Mass lesion with early enhancement, early wash...","enhance and early washout, favor hcc",21
41,Post resection change: s/p left hepatectomy,Post resection change: s/p [localization],8


In [7]:
df.to_excel('output1.xlsx')

In [9]:
#定义删除除字母,数字，汉字以外的所有符号的函数
def remove_punctuation(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line
 
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords  
 
#加载停用词
stopwords = stopwordslist("StopWord.txt")


In [12]:
df['clean_review'] = df['句子'].apply(remove_punctuation)
df.sample(10)

,類別,句子,label_id,clean_review
88,"Mass lesion with early enhancement, early wash...",local liver nodule size with early enhancement...,21,locallivernodulesizewithearlyenhancementearlyw...
129,"Mass lesion with early enhancement, early wash...",an early enhance and early washout mass size n...,21,anearlyenhanceandearlywashoutmasssizenoteinlls...
17,Lobulated contour of liver parenchyma favoring...,compatible with liver cirrhosis,2,compatiblewithlivercirrhosis
224,Mass lesion with no early enhancement but earl...,"early washout, compatible with hcc",22,earlywashoutcompatiblewithhcc
228,Mass lesion with no early enhancement but earl...,"image and tsize, poor early enhancement, mild ...",22,imageandtsizepoorearlyenhancementmildearlywash...
13,Lobulated contour of liver parenchyma favoring...,lobulated contour of liver,2,lobulatedcontourofliver
135,"Mass lesion with early enhancement, early wash...",an early enhance and washout liver nodule in l...,21,anearlyenhanceandwashoutlivernoduleinlocalsize...
166,"Mass lesion with early enhancement, early wash...",size cm early enhancement and early washout no...,21,sizecmearlyenhancementandearlywashoutnoduleinl...
126,"Mass lesion with early enhancement, early wash...",one early enhance liver nodule in local about ...,21,oneearlyenhancelivernoduleinlocalaboutsizewith...
233,"Mass lesion with early enhancement, persistent...",favor hemangioma,23,favorhemangioma


In [18]:
df['cut_review'] = df['clean_review'].apply(lambda x: " ".join([w for w in list(jb.cut(x)) if w not in stopwords]))
df.head()

,類別,句子,label_id,clean_review,cut_review
0,Hypertrophic change of liver,Hypertrophic change of liver,0,Hypertrophicchangeofliver,Hypertrophicchangeofliver
1,Atrophy change of liver,mild cirrhotic change of liver,1,mildcirrhoticchangeofliver,mildcirrhoticchangeofliver
2,Atrophy change of liver,atrophic change of local liver,1,atrophicchangeoflocalliver,atrophicchangeoflocalliver
3,Atrophy change of liver,local liver atrophic change,1,localliveratrophicchange,localliveratrophicchange
4,Atrophy change of liver,atrophic change small of lls and local liver,1,atrophicchangesmallofllsandlocalliver,atrophicchangesmallofllsandlocalliver


In [19]:
# 设置最频繁使用的50000个词
MAX_NB_WORDS = 50000
# 每条cut_review最大的长度
MAX_SEQUENCE_LENGTH = 250
# 设置Embeddingceng层的维度
EMBEDDING_DIM = 100
 
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['cut_review'].values)
word_index = tokenizer.word_index
print('共有 %s 个不相同的词语.' % len(word_index))

NameError: name 'Tokenizer' is not defined